<a href="https://colab.research.google.com/github/rajatguptakgp/practical_machine_learning/blob/main/mixed_precision_training_on_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras import mixed_precision

In [2]:
# connecting Colab to available TPU instance
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)

# TPU initialization
tf.tpu.experimental.initialize_tpu_system(resolver)
tf.config.list_logical_devices('TPU')

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.119.7.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.119.7.234:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


[LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'),
 LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'),
 LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'),
 LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'),
 LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'),
 LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'),
 LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'),
 LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]

In [3]:
# initialize distributed training strategy
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [4]:
with strategy.scope():

  # model architecture
  num_nodes = [1024, 512, 256, 128, 64]
  model_layers = []
  model_layers.append(layers.Flatten(dtype='float32'))
  for nodes in num_nodes:
      model_layers.append(layers.Dense(nodes, activation='relu', dtype='float32'))
  model_layers.append(layers.Activation('softmax', dtype='float32'))

  # building model
  model = Sequential(model_layers)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

  model.build(input_shape = (None, 32, 32, 3))
  model.summary()  

for layer_idx, layer in enumerate(model.layers): 
  print(layer.name)
  print(f'Variable dtype: {layer.dtype}')
  print(f'Computation dtype: {layer.compute_dtype}') 
  print()  

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 1024)              3146752   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 activation (Activation)     (None, 64)                0

In [5]:
# loading CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

# normalizing inputs
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# fitting model
history = model.fit(x_train, y_train, batch_size=2048, epochs=50, validation_split=0.2)
test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', round(test_scores[0], 3))
print('Test accuracy:', round(test_scores[1], 3))

x_train: (50000, 32, 32, 3)
y_train: (50000, 1)
x_test: (10000, 32, 32, 3)
y_test: (10000, 1)
Epoch 1/50
Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.


20/20 [==============================] - 15s 498ms/step - loss: 3.4791 - accuracy: 0.1120 - val_loss: 3.1170 - val_accuracy: 0.1640
Epoch 2/50
20/20 [==============================] - 2s 129ms/step - loss: 3.0144 - accuracy: 0.2082 - val_loss: 2.9505 - val_accuracy: 0.2306
Epoch 3/50
20/20 [==============================] - 2s 106ms/step - loss: 2.9108 - accuracy: 0.2547 - val_loss: 2.8804 - val_accuracy: 0.2737
Epoch 4/50
20/20 [==============================] - 2s 79ms/step - loss: 2.8534 - accuracy: 0.2862 - val_loss: 2.8385 - val_accuracy: 0.3017
Epoch 5/50
20/20 [==============================] - 1s 69ms/step - loss: 2.8059 - accuracy: 0.3024 - val_loss: 2.7814 - val_accuracy: 0.3117
Epoch 6/50
20/20 [==============================] - 1s 69ms/step - loss: 2.7765 - accuracy: 0.3083 - val_loss: 2.7899 - val_accuracy: 0.2946
Epoch 7/50
20/20 [==============================] - 1s 67ms/step - loss: 2.7436 - accuracy: 0.3246 - val_loss: 2.7290 - val_accuracy: 0.3313
Epoch 8/50
20/20 [==

In [6]:
policy = mixed_precision.Policy('mixed_bfloat16')
mixed_precision.set_global_policy(policy)

print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: bfloat16
Variable dtype: float32


In [7]:
with strategy.scope():

  # model architecture
  num_nodes = [1024, 512, 256, 128, 64]
  model_layers = []
  model_layers.append(layers.Flatten())
  for nodes in num_nodes:
      model_layers.append(layers.Dense(nodes, activation='relu'))
  # final layer has to be float32    
  model_layers.append(layers.Activation('softmax', dtype='float32'))

  # building model
  model = Sequential(model_layers)
  model.compile(loss='sparse_categorical_crossentropy', steps_per_execution = 50, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

  model.build(input_shape = (None, 32, 32, 3))
  model.summary()    

for layer_idx, layer in enumerate(model.layers): 
  print(layer.name)
  print(f'Variable dtype: {layer.dtype}')
  print(f'Computation dtype: {layer.compute_dtype}') 
  print() 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_5 (Dense)             (None, 1024)              3146752   
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 activation_1 (Activation)   (None, 64)               

In [8]:
# loading CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

# normalizing inputs
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# fitting model
history = model.fit(x_train, y_train, batch_size=2048, epochs=50, validation_split=0.2)
test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', round(test_scores[0], 3))
print('Test accuracy:', round(test_scores[1], 3))

x_train: (50000, 32, 32, 3)
y_train: (50000, 1)
x_test: (10000, 32, 32, 3)
y_test: (10000, 1)
Epoch 1/50
20/20 [==============================] - 14s 687ms/step - loss: 3.2250 - accuracy: 0.1377 - val_loss: 2.8079 - val_accuracy: 0.2029
Epoch 2/50
20/20 [==============================] - 1s 56ms/step - loss: 2.7320 - accuracy: 0.2533 - val_loss: 2.6581 - val_accuracy: 0.2742
Epoch 3/50
20/20 [==============================] - 1s 55ms/step - loss: 2.6034 - accuracy: 0.3042 - val_loss: 2.5525 - val_accuracy: 0.3171
Epoch 4/50
20/20 [==============================] - 1s 56ms/step - loss: 2.5268 - accuracy: 0.3295 - val_loss: 2.4951 - val_accuracy: 0.3350
Epoch 5/50
20/20 [==============================] - 1s 54ms/step - loss: 2.4731 - accuracy: 0.3458 - val_loss: 2.4465 - val_accuracy: 0.3486
Epoch 6/50
20/20 [==============================] - 1s 56ms/step - loss: 2.4268 - accuracy: 0.3606 - val_loss: 2.4030 - val_accuracy: 0.3639
Epoch 7/50
20/20 [==============================] - 1s 53m